somatic değişkenini en iyi açıklayan değişkenler hangileri? 

In [2]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso 
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt 

df = pd.read_csv("final_learning_potential_dataset.csv")

print(df.shape)
df.head()

(427, 7)


,stress_congeneric_score,control_congeneric,self_efficacy_congeneric,depression_congeneric_score,somatic_congeneric,cog_affect_congeneric,learning_potential_score
0,-0.994499,0.568381,1.568341,-0.852725,-1.522905,-0.294627,2.153331
1,-1.601830,1.258011,1.846971,-1.430514,-1.403349,-1.390766,3.369578
2,0.183967,-0.516081,0.397377,-0.347319,0.019725,-0.594503,0.234547
3,0.419527,-0.581236,-0.063644,-0.825759,-1.003940,-0.594503,0.534051
4,-1.093718,1.413048,0.214986,0.870991,1.372580,0.501635,-0.020603


predictors ve target seçimi 

In [4]:
target_col = "somatic_congeneric"

predictor_cols=  [
    "stress_congeneric_score",
    "control_congeneric",
    "self_efficacy_congeneric",
    "depression_congeneric_score",
    "cog_affect_congeneric"
]

6 değişkenli PCA ile hesaplandığı için data leakage sebebiyle learning_potential_score, predictors bünyesine alınmadı 

In [13]:
model_df = df[[target_col] + predictor_cols].dropna()

X = model_df[predictor_cols]
y = model_df[target_col]

X.head(), y.head()
X.shape, y.shape

((427, 5), (427,))

train test ayrımı 